In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import os
import random
import seaborn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3,MobileNetV2
from tensorflow.keras.models import Model,save_model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, BatchNormalization,Dropout,Input,Average

In [2]:
data_dir =r"E:\1. Thesis\Update_Code\Dataset"
keyword= ['Anthracnose','Bacterial Canker','Cutting Weevil','Die Back','Gall Midge','Healthy','Powdery Mildew','Sooty Mould']

In [3]:
class_folders=os.listdir(data_dir)
image_paths=[]
labels=[]

for class_folder in class_folders:
    for key in keyword:
        if key in class_folder:
                class_path=os.path.join(data_dir,class_folder)
                image_files=os.listdir(class_path)
                for image_file in image_files:
                    image_path=os.path.join(class_path,image_file)
                    image_paths.append(image_path)
                    labels.append(class_folder)
    
df=pd.DataFrame({'image_path':image_paths,'label':labels})
print("The classes:",np.unique(df['label']))

class_counts=df['label'].value_counts()
class_counts

The classes: ['Anthracnose' 'Bacterial Canker' 'Cutting Weevil' 'Die Back' 'Gall Midge'
 'Healthy' 'Powdery Mildew' 'Sooty Mould']


label
Anthracnose         500
Bacterial Canker    500
Cutting Weevil      500
Die Back            500
Gall Midge          500
Healthy             500
Powdery Mildew      500
Sooty Mould         500
Name: count, dtype: int64

In [4]:
#split train(60%) and the rest(40%)
train_df, rest_df = train_test_split(df,test_size=0.4, random_state=42)

#split valiidation(50%) and test(50%)
val_df, test_df = train_test_split(rest_df, test_size=0.5, random_state=42)

In [5]:
def load_images_for_cnn(train_df, 
                        val_df, 
                        test_df, 
                        batch_size=32, 
                        target_size=(224,224)):

    train_datagen=ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    train_generator=train_datagen.flow_from_dataframe(
        train_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    validation_datagen=ImageDataGenerator(rescale=1./255)
    validation_generator = validation_datagen.flow_from_dataframe(
        val_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    test_datagen=ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        test_df,
        x_col='image_path',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    return train_generator,validation_generator,test_generator

train_generator,validation_generator,test_generator=load_images_for_cnn(train_df,
                                                                        val_df, 
                                                                        test_df, 
                                                                        batch_size=32, 
                                                                        target_size=(224,224))

Found 2400 validated image filenames belonging to 8 classes.
Found 800 validated image filenames belonging to 8 classes.
Found 800 validated image filenames belonging to 8 classes.


In [6]:
# Load Models
model_path1 = r"E:\1. Thesis\Update_Code\SavedModel\DenseNet1213F.hdf5"
m1 = load_model(model_path1)
m1 = Model(inputs=m1.inputs, outputs=m1.outputs, name='DenseNet121') 

model_path2 = r"E:\1. Thesis\Update_Code\SavedModel\ResNet152V2.hdf5"
m2 = load_model(model_path2)
m2 = Model(inputs=m2.inputs, outputs=m2.outputs, name='ResNet152V2')

model_path3 = r"E:\1. Thesis\Update_Code\SavedModel\InceptionV3F.hdf5"
m3 = load_model(model_path3)
m3 = Model(inputs=m3.inputs, outputs=m3.outputs, name='InceptionV3') 


models=[m1,m2,m3]

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Average

models = [m1, m2, m3]
model_input = Input(shape=(224,224,3), name="ensemble_input")

# Ensure model outputs are single tensors, not lists
model_outputs = [model(model_input) for model in models]

# If any output is wrapped in a list/tuple, extract the first tensor
model_outputs = [output[0] if isinstance(output, (list, tuple)) else output for output in model_outputs]

# Merge outputs
ensemble_output = Average()(model_outputs)

# Create ensemble model
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)


In [20]:
print([type(output) for output in model_outputs])
print([output.shape for output in model_outputs])


[<class 'keras.src.backend.common.keras_tensor.KerasTensor'>, <class 'keras.src.backend.common.keras_tensor.KerasTensor'>, <class 'keras.src.backend.common.keras_tensor.KerasTensor'>]
[(None, 8), (None, 8), (None, 8)]


In [21]:
# Compile Model
ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

In [22]:
# Train Model
epochs = 10
training_history = ensemble_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

C:\Users\Sanzida\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


C:\Users\Sanzida\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
C:\Users\Sanzida\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_2']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)
C:\Users\Sanzida\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1808s 23s/step - accuracy: 0.9701 - loss: 0.2707 - val_accuracy: 0.9725 - val_loss: 0.2167
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1673s 22s/step - accuracy: 0.9681 - loss: 0.2429 - val_accuracy: 0.9762 - val_loss: 0.1933
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1750s 23s/step - accuracy: 0.9782 - loss: 0.2097 - val_accuracy: 0.9750 - val_loss: 0.1749
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1535s 20s/step - accuracy: 0.9831 - loss: 0.1894 - val_accuracy: 0.9775 - val_loss: 0.1591
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1395s 19s/step - accuracy: 0.9788 - loss: 0.1743 - val_accuracy: 0.9787 - val_loss: 0.1467
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1372s 18s/step - accuracy: 0.9799 - loss: 0.1677 - val_accuracy: 0.9887 - val_loss: 0.1331
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1546s 21s/step - accuracy: 0.9825 - loss: 0.1504 - val_accuracy: 0.9800 - val_loss: 0.1288
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1342s 18s/step - accuracy: 0.9809 - loss: 0.1460 - val_accuracy: 0.986

In [25]:
class WeightedAverageLayer(tf.keras.layers.Layer):  # Corrected Layer inheritance
    def __init__(self, w1, w2, w3, **kwargs):  # Corrected constructor
        super(WeightedAverageLayer, self).__init__(**kwargs)  # Corrected super call
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]


In [26]:
ensemble_output=WeightedAverageLayer(0.6,0.3,0.1)(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)

In [27]:
# Compile Model
ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

In [28]:
# Train Model
epochs = 10
training_history = ensemble_model.fit(train_generator, 
                                      epochs=epochs, 
                                      validation_data=validation_generator)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1693s 20s/step - accuracy: 0.9877 - loss: 0.1308 - val_accuracy: 0.9875 - val_loss: 0.0987
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1370s 18s/step - accuracy: 0.9906 - loss: 0.1165 - val_accuracy: 0.9887 - val_loss: 0.0913
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1353s 18s/step - accuracy: 0.9889 - loss: 0.1191 - val_accuracy: 0.9900 - val_loss: 0.0855
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 6054s 82s/step - accuracy: 0.9867 - loss: 0.1096 - val_accuracy: 0.9912 - val_loss: 0.0803
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 9191s 124s/step - accuracy: 0.9911 - loss: 0.1038 - val_accuracy: 0.9900 - val_loss: 0.0775
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1383s 18s/step - accuracy: 0.9911 - loss: 0.0961 - val_accuracy: 0.9925 - val_loss: 0.0738
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3998s 54s/step - accuracy: 0.9904 - loss: 0.1007 - val_accuracy: 0.9937 - val_loss: 0.0693
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2310s 31s/step - accuracy: 0.9934 - loss: 0.0847 - val_acc